In [1]:
!pip install cloudscraper
from bs4 import BeautifulSoup
import cloudscraper
import csv
import re
import time
import pandas as pd

Defaulting to user installation because normal site-packages is not writeable


In [2]:
links_file_1 = pd.read_csv('links_file_1.csv')

In [3]:
def get_title(soup):
    title_and_views = soup.find('h2')
    if title_and_views is None:
        return 'PLACEHOLDER', 'PLACEHOLDER'
    title_parts = title_and_views.text.strip().split(' - ')
    return title_parts[0], title_parts[-2] if len(title_parts) >= 2 else 'PLACEHOLDER'


In [4]:
def get_body(soup):
    body = soup.find('div', {'class': 'narrow'})
    return body.text.strip() if body is not None else 'PLACEHOLDER'


In [5]:
def get_comments_and_count(soup, scraper, url):
    all_comments = []
    previous_comments = None
    page = 0
    while True:
        current_url = f"{url}/{page}" if page > 0 else url
        raw_html = scraper.get(current_url).text
        soup = BeautifulSoup(raw_html, 'html.parser')

        comments = []
        for div in soup.find_all('div', class_='narrow'):
            if div.blockquote:
                div.blockquote.extract()
            comments.append(div.text.strip())

        if comments == previous_comments:
            break

        all_comments.extend(comments)
        previous_comments = comments
        page += 1

    comments_joined = ';;;'.join(all_comments)
    comment_count = len(all_comments)
    return comments_joined, comment_count


In [6]:
def get_total_views(soup):
    views_element = soup.find('p', class_='bold')
    if views_element:
        views_text = views_element.text.strip()
        views_match = re.search(r'\((\d{1,3}(,\d{3})*|\d+)\s*Views\)', views_text)
        if views_match:
            return int(views_match.group(1).replace(',', ''))
    return 0


In [7]:
# Define the filename and the header of the CSV file
filename = 'Nairaland Dataset Part1.csv'
header = ['Title', 'Body', 'Category', 'Comments', 'Comment Count', 'Time Posted', 'Date Posted', 'Total Views', 'Url']

In [ ]:
# Loop through the list of URLs and write the data to the CSV file
with open(filename, mode='a', encoding='utf-8', newline='') as file:
   # writer = csv.writer(file)
    writer = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
  
    writer.writerow(header)
    counter = 0  # Counts the number of iterations

    for index, row in links_file_1.iterrows():
        url = row['Url']
        post_time = row['Time posted']
        post_date = row['Date posted']
        
        scraper = cloudscraper.create_scraper()
        
        try:  # Add try block here
            raw_text = scraper.get(url).text
            soup = BeautifulSoup(raw_text, 'html.parser')

            Title, Category = get_title(soup)
            Body = get_body(soup)
            comments_joined, comment_count = get_comments_and_count(soup, scraper, url)
            total_views = get_total_views(soup)

            data = [Title, Body, Category, comments_joined, comment_count, post_time, post_date, total_views, url]
            writer.writerow(data)
            
        
        except Exception as e:  # Add except block here
            print(f"Error occurred while processing {url}: {e}")


        # Add delay after every 3 iterations to avoid overloading the server
        if counter % 3 == 0 and counter != 0:
            print(f"Pausing for 10 seconds after {counter} iterations")
            time.sleep(10)
        counter += 1
        
                                                                                                                             

Pausing for 10 seconds after 3 iterations
